In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
import joblib  # Biblioteca para salvar o modelo

def prepare_data(dataset_path, forecast_days=30, threshold=0.03):
    """
    Prepara os dados para treinamento e teste.
    """
    # Carregar o dataset
    dataset = pd.read_csv(dataset_path)
    
    # Criar variável alvo baseada no retorno futuro
    dataset['Future_Return'] = (dataset['Close/Last'] - dataset['Close/Last'].shift(-forecast_days) ) / dataset['Close/Last'].shift(-forecast_days)
    dataset['Target'] = (dataset['Future_Return'] > threshold).astype(int)

    # Selecionar features e variável alvo
    X = dataset[['High', 'Low', 'Volume', 'SMA_20', 'Close/Last', 'SMA_50', 'Volatility_30', 'Avg_Volume_30']].values
    y = dataset['Target'].values

    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return dataset, X_train, X_test, y_train, y_test

def train_model(X_train, y_train, model_path='xgboost_model.joblib'):
    """
    Treina o modelo XGBoost e salva o modelo em um arquivo.
    """
    model = XGBClassifier(eval_metric='logloss')
    model.fit(X_train, y_train)
    
    # Salvar o modelo em um arquivo
    joblib.dump(model, model_path)
    print(f"Modelo salvo em: {model_path}")
    
    return model

def predict_model(model, X_test):
    """
    Faz previsões usando o modelo treinado.
    """
    return model.predict(X_test)

# Caminho do dataset
dataset_path = 'base_tratada.csv'
model_save_path = 'xgboost_model.joblib'

# Preparar dados
dataset, X_train, X_test, y_train, y_test = prepare_data(dataset_path)

# Treinar modelo
model = train_model(X_train, y_train, model_save_path)

# Fazer previsões
y_pred = predict_model(model, X_test)

# Avaliar modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Faz previsão de todas os dias do dataset
dataset['prediction'] = predict_model(model, dataset[['High', 'Low', 'Volume', 'SMA_20', 'Close/Last', 'SMA_50', 'Volatility_30', 'Avg_Volume_30']])

#Remover linhas com datas inválidas
dataset.dropna(inplace=True)

#Salva dataset para csv com as predições
dataset.to_csv('model_output.csv', index=False)

Modelo salvo em: xgboost_model.joblib
Acurácia do modelo: 0.91

Matriz de Confusão:
[[271  15]
 [ 22  89]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       286
           1       0.86      0.80      0.83       111

    accuracy                           0.91       397
   macro avg       0.89      0.87      0.88       397
weighted avg       0.91      0.91      0.91       397

